In [89]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [90]:
sparql.setQuery("""
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q16521.
  ?item wdt:P171* wd:Q5113.
  ?item wdt:P105 wd:Q7432.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru". }
}""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [91]:
import time
start_time=None
def start_time():
    global start_time
    start_time=time.time()
def next_time():
    elapsed=time.time()-start_time
    start_time()
    return elapsed

In [92]:
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['item.value', 'itemLabel.value']].head()

,item.value,itemLabel.value
0,http://www.wikidata.org/entity/Q19056,Канадская дикуша
1,http://www.wikidata.org/entity/Q26691,Краснозобая гагара
2,http://www.wikidata.org/entity/Q80832,Pterocles senegallus
3,http://www.wikidata.org/entity/Q127880,Pterocles quadricinctus
4,http://www.wikidata.org/entity/Q127881,Ночной рябок


In [105]:


#Get all P for given Q
#qq("Q25402")
def qq(qid):
    item = pywikibot.ItemPage(repo, qid)
    item_dict = item.get()
    clm_dict = item_dict["claims"]
    return list(clm_dict.keys())

#Get name of given P
#pname("P2050")
def pname(pid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel 
    WHERE 
    {
      ?item wdt:* wd:"""+pid+""".
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }""")
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df_p = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df_p[['item.value', 'itemLabel.value']].head().values.tolist()[0][1]

keep=results_df[['item.value', 'itemLabel.value']].values.tolist()
count=0
for i,k in enumerate(keep):
    if i % 100 == 0:
        print(i,"\t",i/len(keep)*100)
    qqid=k[0].split('/')[-1]
    P_list=qq(qqid)
    if 'P2050' in P_list:
       count+=1  #24
    #list_p=[pname(p) for p in q_0]
    #print(list_p)
print(count)


0 	 0.0
100 	 0.5255965520866183
200 	 1.0511931041732365
300 	 1.5767896562598551
400 	 2.102386208346473
500 	 2.627982760433092
600 	 3.1535793125197102
700 	 3.679175864606328
800 	 4.204772416692946
900 	 4.730368968779565
1000 	 5.255965520866184
1100 	 5.781562072952801
1200 	 6.3071586250394205
1300 	 6.832755177126037
1400 	 7.358351729212656
1500 	 7.883948281299275
1600 	 8.409544833385892
1700 	 8.935141385472512
1800 	 9.46073793755913
1900 	 9.986334489645749
2000 	 10.511931041732367
2100 	 11.037527593818984
2200 	 11.563124145905602
2300 	 12.08872069799222
2400 	 12.614317250078841
2500 	 13.139913802165456
2600 	 13.665510354252074
2700 	 14.191106906338694
2800 	 14.716703458425313
2900 	 15.242300010511931
3000 	 15.76789656259855
3100 	 16.293493114685166
3200 	 16.819089666771784
3300 	 17.344686218858403
3400 	 17.870282770945025
3500 	 18.395879323031643
3600 	 18.92147587511826
3700 	 19.44707242720488
3800 	 19.972668979291498
3900 	 20.498265531378117
4000 	

In [107]:
count/len(keep)*100 #0.12%, (24 items)

0.12614317250078838

In [45]:
import urllib.request
keep=results_df[['item.value', 'itemLabel.value']].values.tolist()
q_len=len(keep)
q=[elem[0].split('/')[-1] for elem in keep[:]]
#print('Q25402' in q)

def level(n):
    return n/q_len*100.0

25.0

In [52]:
keep=results_df[['item.value', 'itemLabel.value']]
QNums=[]
lvlstep=1
currlvl=lvlstep
for elem in keep[['item.value']].values.tolist():
    Qnum=elem[0].split('/')[-1]
    #if Qnum=='Q25402':
        #print(Qnum)
    with urllib.request.urlopen('https://www.wikidata.org/wiki/'+Qnum) as response:
        html = str(response.read())
    b=html.split('Property:')[1:]
    lvl=0
    for i in b:
        i=i.split("</a>")[0]
        if '">' in i:
            lvl+=1
            el=(i.split('">'))[1]
            if el=='wingspan':
                QNums+=[Qnum]
            
            if level(lvl)>=currlvl:
                currlvl+=lvlstep
                print(level(lvl)," %",end="\t")

1.003889414485441  %	

KeyboardInterrupt: 

In [10]:
import pywikibot
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

In [ ]:
item = pywikibot.ItemPage(repo, "Q647138") #Q5113
item_dict = item.get()


In [81]:
sparql.setQuery("""
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:* wd:P2050.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df[['item.value', 'itemLabel.value']].head()

,item.value,itemLabel.value
0,http://www.wikidata.org/entity/P2050,wingspan


In [59]:
item = pywikibot.ItemPage(repo, "Q25402") #Q5113
item_dict = item.get()
item_dict

{'aliases': {'pl': ['Cygnus olor'],
  'eu': ['Cygnus olor'],
  'es': ['Cisne común',
   'Cisne comun',
   'Cisne vulgar',
   'Cisne blanco',
   'Cisne mudo'],
  'af': ['C. olor', 'Witswaan'],
  'oc': ['Cicne mut'],
  'hu': ['Cygnus olor'],
  'et': ['Cygnus olor'],
  'br': ['Cygnus olor'],
  'nl': ['Cygnus olor'],
  'ar': ['التم الصامت'],
  'sv': ['Cygnus olor'],
  'pt': ['Cisne branco', 'Cygnus olor', 'Cisne-vulgar'],
  'eo': ['Cygnus olor'],
  'is': ['Cygnus olor'],
  'ru': ['Шипун', 'Cygnus olor'],
  'sr': ['Cygnus olor', 'Лабуд грбац'],
  'tr': ['Cygnus olor', 'Anas olor'],
  'fi': ['Cygnus olor'],
  'uk': ['Лебідь білий', 'Cygnus olor'],
  'nn': ['Cygnus olor'],
  'az': ['Sthenelides olor',
   'Fısıldayan qu quşu',
   'Fısıldayan qu',
   'Cygnus immutabilis',
   'Anas olor Gmelin',
   'Cygnus olor'],
  'hr': ['Cygnus olor'],
  'kk': ['Cygnus olor'],
  'da': ['Cygnus olor', 'Knopsvanen'],
  'gsw': ['Schwan'],
  'be': ['Cygnus olor'],
  'he': ['ברבור אילם'],
  'fr': ['Cygnus Immutabi

In [116]:
def pname(pid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel 
    WHERE 
    {
      ?item wdt:* wd:"""+pid+""".
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }""")
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df_p = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df_p[['item.value', 'itemLabel.value']].head().values.tolist()[0][1]

with open("fetchprops.txt") as f:
    d=f.read()
allp=set([i for i in d.replace('\n','\t').split('\t') if i.startswith("P")])
allp=list(allp)
Prelated={}
for a in allp:
    Prelated[a]=pname(a)


In [119]:
u=[]
for p in Prelated:
    u+=[p]
    u+=["\t"]
    u+=[Prelated[p]]
with open("fetchpropnames.txt","w",encoding="utf-8") as f:
    f.write("\n".join(u))